In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.0 MB/s eta 0:00:00


In [2]:
import re
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
from google.colab import drive
import pandas as pd
from datetime import datetime
from pandas.io.formats.info import pprint_thing
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
season_duration = pd.read_excel('/content/drive/MyDrive/DS105L/author_credit_nba/season_duration_1.xlsx')
season_duration_range = season_duration['duration'].to_list()
duration = []
for durations in season_duration_range:
  durations = durations.replace('\xa0',' ')
  durations = durations.replace('-', '–')
  start_date_str, end_date_str = durations.split(" – ")
  end_date_str = end_date_str[:-1] + re.sub('[^0-9]','', end_date_str[-1])
  
  # Parse the start and end dates into datetime objects
  start_date = datetime.strptime(start_date_str, "%B %d, %Y")
  end_date = datetime.strptime(end_date_str, "%B %d, %Y")

  # Format the start and end dates as strings with numerical month and year
  start_date_num = start_date.strftime("%Y%m%d")
  end_date_num = end_date.strftime("%Y%m%d")

  duration.append([int(start_date_num), int(end_date_num)])

season_duration['duration'] = duration

display(season_duration)

,season,duration
0,1950-51,"[19501031, 19510318]"
1,1951-52,"[19511101, 19520316]"
2,1952-53,"[19521031, 19530317]"
3,1953-54,"[19531030, 19540314]"
4,1954-55,"[19541030, 19550314]"
...,...,...
68,2018-19,"[20181016, 20190410]"
69,2019-20,"[20191022, 20200311]"
70,2020-21,"[20201222, 20210516]"
71,2021-22,"[20211019, 20220410]"


In [6]:
archive_0_600 = pd.read_csv('/content/drive/MyDrive/DS105L/author_credit_nba/news_600_author_credit_1.csv')
archive_600_1200 = pd.read_csv('/content/drive/MyDrive/DS105L/author_credit_nba/news_600_author_credit_1.csv')
archive_1200_1640 = pd.read_csv('/content/drive/MyDrive/DS105L/author_credit_nba/news_600_author_credit_1.csv')

def date_season_transfer(dataframe):

  time_list = dataframe['time'].to_list()
  date_list = []

  for date in time_list:
    date_obj = datetime.strptime(date, '%B %d, %Y')
    numerical_date = date_obj.strftime('%Y%m%d')
    date_list.append(int(numerical_date))

  season_duration_list = list(season_duration.iloc[:,1])

  season_transferred = []
  for new_date in date_list:
    found_season = False
    for period in duration:
      if period[0] <= new_date <= period[1]:
        row_index = [x[0] for x in season_duration_list].index(period[0])
        #row_index,column_index = pd.DataFrame(season_duration.iloc[:,1][0] == period[0])
        #print(row_index)
        season = season_duration.iloc[row_index,0]
        #print(season)
        season_transferred.append(season)
        found_season = True
        break
    if found_season == False:
      #print(f"No season found for news date {news_date}")
      season_transferred.append('off-season news')
      
  dataframe['time'] = season_transferred

  return dataframe

drive.mount('/content/drive')

df1 = date_season_transfer(archive_0_600)
df2 = date_season_transfer(archive_600_1200)
df3 = date_season_transfer(archive_1200_1640)

df1.to_csv('/content/drive/MyDrive/DS105L/author_credit_nba/nba_600_author_credit_season.csv')
df2.to_csv('/content/drive/MyDrive/DS105L/author_credit_nba/nba_1200_author_credit_season.csv')
df3.to_csv('/content/drive/MyDrive/DS105L/author_credit_nba/nba_1600_author_credit_season.csv')

display(df1)
display(df2)
display(df3)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,title,time,url,author,credit_article,credit_followers,credit_tweets
0,0,"Bucks’ George Hill Returns To Milwaukee, Encou...",off-season news,http://global.nba.com/news/bucks-george-hill-r...,Steve Megargee,124794,1906,34099
1,1,"NBA, WNBA Ramp Up Efforts To Get Out The Vote ...",off-season news,http://global.nba.com/news/nba-wnba-ramp-up-ef...,Brian Martin,220,216,1172
2,2,"Pat Riley Discusses Heat Future, Plans For Kee...",off-season news,http://global.nba.com/news/pat-riley-discusses...,Tim Reynolds | Associated Press,None,None,None
3,3,Report: Amar’e Stoudemire Joins Steve Nash’s C...,off-season news,http://global.nba.com/news/report-amare-stoude...,NBA News,Default,Default,Default
4,4,ESPN To Host Virtual 2020 NBA Draft Presented ...,off-season news,http://global.nba.com/news/espn-to-host-virtua...,Official Release,Default,Default,Default
...,...,...,...,...,...,...,...,...
5995,5995,"Antetokounmpo Scores 34, Bucks Top Spurs 135-129",2018-19,http://global.nba.com/news/antetokounmpo-score...,NBA News,Default,Default,Default
5996,5996,Mavericks Pull Away In 4th For 113-104 Win Ove...,2018-19,http://global.nba.com/news/mavericks-pull-away...,NBA News,Default,Default,Default
5997,5997,"Murray Leads Nuggets Past Thunder, 105-98",2018-19,http://global.nba.com/news/murray-leads-nugget...,NBA News,Default,Default,Default
5998,5998,"Towns’ 35 Points, 22 Boards Lead Wolves Over B...",2018-19,http://global.nba.com/news/towns-35-points-22-...,NBA News,Default,Default,Default


,Unnamed: 0,title,time,url,author,credit_article,credit_followers,credit_tweets
0,0,"Bucks’ George Hill Returns To Milwaukee, Encou...",off-season news,http://global.nba.com/news/bucks-george-hill-r...,Steve Megargee,124794,1906,34099
1,1,"NBA, WNBA Ramp Up Efforts To Get Out The Vote ...",off-season news,http://global.nba.com/news/nba-wnba-ramp-up-ef...,Brian Martin,220,216,1172
2,2,"Pat Riley Discusses Heat Future, Plans For Kee...",off-season news,http://global.nba.com/news/pat-riley-discusses...,Tim Reynolds | Associated Press,None,None,None
3,3,Report: Amar’e Stoudemire Joins Steve Nash’s C...,off-season news,http://global.nba.com/news/report-amare-stoude...,NBA News,Default,Default,Default
4,4,ESPN To Host Virtual 2020 NBA Draft Presented ...,off-season news,http://global.nba.com/news/espn-to-host-virtua...,Official Release,Default,Default,Default
...,...,...,...,...,...,...,...,...
5995,5995,"Antetokounmpo Scores 34, Bucks Top Spurs 135-129",2018-19,http://global.nba.com/news/antetokounmpo-score...,NBA News,Default,Default,Default
5996,5996,Mavericks Pull Away In 4th For 113-104 Win Ove...,2018-19,http://global.nba.com/news/mavericks-pull-away...,NBA News,Default,Default,Default
5997,5997,"Murray Leads Nuggets Past Thunder, 105-98",2018-19,http://global.nba.com/news/murray-leads-nugget...,NBA News,Default,Default,Default
5998,5998,"Towns’ 35 Points, 22 Boards Lead Wolves Over B...",2018-19,http://global.nba.com/news/towns-35-points-22-...,NBA News,Default,Default,Default


,Unnamed: 0,title,time,url,author,credit_article,credit_followers,credit_tweets
0,0,"Bucks’ George Hill Returns To Milwaukee, Encou...",off-season news,http://global.nba.com/news/bucks-george-hill-r...,Steve Megargee,124794,1906,34099
1,1,"NBA, WNBA Ramp Up Efforts To Get Out The Vote ...",off-season news,http://global.nba.com/news/nba-wnba-ramp-up-ef...,Brian Martin,220,216,1172
2,2,"Pat Riley Discusses Heat Future, Plans For Kee...",off-season news,http://global.nba.com/news/pat-riley-discusses...,Tim Reynolds | Associated Press,None,None,None
3,3,Report: Amar’e Stoudemire Joins Steve Nash’s C...,off-season news,http://global.nba.com/news/report-amare-stoude...,NBA News,Default,Default,Default
4,4,ESPN To Host Virtual 2020 NBA Draft Presented ...,off-season news,http://global.nba.com/news/espn-to-host-virtua...,Official Release,Default,Default,Default
...,...,...,...,...,...,...,...,...
5995,5995,"Antetokounmpo Scores 34, Bucks Top Spurs 135-129",2018-19,http://global.nba.com/news/antetokounmpo-score...,NBA News,Default,Default,Default
5996,5996,Mavericks Pull Away In 4th For 113-104 Win Ove...,2018-19,http://global.nba.com/news/mavericks-pull-away...,NBA News,Default,Default,Default
5997,5997,"Murray Leads Nuggets Past Thunder, 105-98",2018-19,http://global.nba.com/news/murray-leads-nugget...,NBA News,Default,Default,Default
5998,5998,"Towns’ 35 Points, 22 Boards Lead Wolves Over B...",2018-19,http://global.nba.com/news/towns-35-points-22-...,NBA News,Default,Default,Default


In [ ]:
pd.read_csv('/content/drive/MyDrive/DS105L/author_credit_nba/news_600_credit_1.csv')